In [ ]:
from pygraphblas import *
from _pygraphblas import lib
import pygraphblas.descriptor
import csv
import sys
import logging
import glob
import operator
from data_loader import DataLoader

In [ ]:
# Setup logger
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)-5s %(message)s'))
log = logging.getLogger(__name__)
log.propagate = False
log.addHandler(handler)
log.setLevel(logging.INFO)

In [ ]:
data_dir = 'sf1k-converted/'
loader = DataLoader(data_dir)

vertices, mappings, matrices = loader.load_all_csvs()

In [ ]:
def MemberFriends(t):
    tagID = mappings['tag'][t]
    tagVector = Vector.from_type(BOOL,matrices['hasTag'].ncols)
    tagVector[tagID] = True
    relevantForumsVector = tagVector.vxm(matrices['hasTag'].transpose())
    relevantPeopleVector = relevantForumsVector.vxm(matrices['hasMember'])
    
    resultMatrixLists = []
    for person in relevantPeopleVector:
        personVector = Vector.from_type(BOOL,relevantPeopleVector.size)
        personVector[person[0]] = True
        personKnowsVector = personVector.vxm(matrices['knows'])
        personKnowsVector *= relevantPeopleVector #only the relevant people are needed
        resultMatrixLists += [[person[0],personKnowsVector.to_lists()[0]]]
    
    resultMatrix = Matrix.from_type(BOOL,relevantPeopleVector.size,relevantPeopleVector.size)
    for entry in resultMatrixLists:
        for value in entry[1]:
            resultMatrix[entry[0],value] = True
    return resultMatrix

In [ ]:
#'Charles_Dickens'.id = 7380
m = MemberFriends(7380)